$WX = y$ <br>

$ W $ = matriz de pesos <br>
$ X $ = matriz de embeddings <br>
$ Y $ = matriz de resultados <br>

En el presente trabajo se analizaron 3 algoritmos distintos con el objetivo de calcular los pesos de la matriz W. <br>

Se despejó la matriz $W$ calculando la Pseudo-Inversa de la matriz de embeddings $(X^{+})$, pero utilizando diferentes descomposiciones matriciales.

$W = YX^{+}$ <br>

#### Importamos librerías

In [2]:
import importlib
import alc
importlib.reload(alc)
import alcModulosNUMPY
importlib.reload(alcModulosNUMPY)

from alc import cargarDataset

#### Lectura de Datos y separación de casos Train y Validation

In [3]:
Xt, Yt, Xv, Yv = cargarDataset("template-alumnos/cats_and_dogs")

---

A continuación, antes de presentar el punto 6, se calcula W mediante los 3 algortimos elaborados.

#### **Algoritmo 2**

In [4]:
#Cálculo de la SVD de x CON NUMPY:
import numpy as np

U, S, Vt = np.linalg.svd(Xt)

n = Xt.shape[0]
m = Xt.shape[1]

Sigma = np.zeros((n, m))

# Colocar los valores singulares en la diagonal
# La cantidad de valores singulares es min(n, m)
for i in range(len(S)):
    Sigma[i, i] = S[i]



In [ ]:
#Cálculo de la SVD de x:
from alcModulosNUMPY import svd_reducida

U, S, Vt = svd_reducida(Xt, k="max")

In [5]:
# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
from alc import pinvSVD

W_SVD = pinvSVD(U, Sigma, Vt, Yt)

---

### 6. Evaluación y Benchmarking
a. Para cada método de resolución de la W de los items anteriores, <br>
generar una matriz de confusión evaluando a partir de los pares de embeddings de validación o testing (Xv, Yv).

In [32]:
def accuracy(tp,tn,fp,fn):
    acertados = tp + tn
    total = tp + tn + fp + fn

    return ((acertados/total)*100)

In [34]:
def matriz_confusion(Y_pred):
    tp = 0  # Predice gato y era gato
    fp = 0  
    tn = 0  # Predice perro y era perro
    fn = 0  

    n = Y_pred.shape[1]
    for i in range(n):
        # aproximamois al que más se acerque ya que nunca va a dar exactamente valores pert a {0,1}
        # 0 = gato, 1 = perro
        if Y_pred[0, i] >= Y_pred[1, i]:
            pred = 0
        else:
            pred = 1

        if Yv[0, i] == 1:
            real = 0
        else:
            real = 1

        if pred == 0 and real == 0:
            tp += 1
        elif pred == 0 and real == 1:
            fp += 1
        elif pred == 1 and real == 1:
            tn += 1
        elif pred == 1 and real == 0:
            fn += 1

    return tp, tn, fp, fn


#### Algoritmo 1 - Ecuaciones Normales

In [ ]:
#Calculamos predicciones de Y
Y_pred_Cholesky = W_cholesky @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_Cholesky)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 1 =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracySVD = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 1 =",accuracySVD,"%")

#### Algoritmo 2 - SVD

In [35]:
#Calculamos predicciones de Y
Y_pred_SVD = W_SVD @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_SVD)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 2 =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracySVD = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 2 =",accuracySVD,"%")

Matriz de confusión Algoritmo 2 =
[[334, 166]]
[[150, 350]]
Accuracy Algoritmo 2 = 68.4 %


#### Algoritmo 3 - Descomposición QR

---

### 7. Síntesis Final

| This | is   |
|------|------|
|   a  | table|